In [1]:
import pandas as pd

In [2]:
df_temp = pd.read_csv('Consumer_Complaints.csv')

In [3]:
df_temp = df_temp[~df_temp['Consumer complaint narrative'].isnull()]

In [4]:
features = df_temp[['Product', 'Consumer complaint narrative']]

In [5]:
features.columns = ['product', 'text']

In [6]:
features.loc[:, 'target'] = features['product'].factorize()[0].copy()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
category_id_df = features[['product', 'target']].drop_duplicates().sort_values('target')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['target', 'product']].values)

In [48]:
dict(zip(category_id_df['target'].values, category_id_df['product'].values))

{0: 'Credit reporting',
 1: 'Consumer Loan',
 2: 'Debt collection',
 3: 'Mortgage',
 4: 'Credit card',
 5: 'Other financial service',
 6: 'Bank account or service',
 7: 'Student loan',
 8: 'Money transfers',
 9: 'Payday loan',
 10: 'Prepaid card',
 11: 'Money transfer',
 12: 'Personal consumer reports',
 13: 'Checking or savings account',
 14: 'Vehicle loan or lease',
 15: 'Credit card or prepaid card',
 16: 'Virtual currency',
 17: 'All kindes of loans'}

In [8]:
category_id_df.loc[category_id_df['target'] == 11, 'product'] = 'Money transfer'

In [9]:
category_id_df.loc[category_id_df['target'] == 12, 'product'] = 'Personal consumer reports'

In [10]:
category_id_df.loc[category_id_df['target'] == 17, 'product'] = 'All kindes of loans'

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

(X_train, X_test, y_train, y_test) = train_test_split(features['text'], features['target'], 
                                                      test_size=0.25, stratify=features['target'])

vect = TfidfVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [31]:
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [32]:
predictions = model.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss
print('ACCURACY:', accuracy_score(predictions, target_test))
print('PRECISION:', precision_score(predictions, target_test, average='macro'))
print('RECALL:', recall_score(predictions, target_test, average='macro'))

ACCURACY: 0.1417794698251551
PRECISION: 0.05437868418616179
RECALL: 0.054572591110018026


In [44]:
model.predict(vect.transform(["I received a letter from Afni stating that I owed {$34.00} due to the service at XXXX. I have NEVER ever signed up for phone services with XXXX nor did I ever sign up for any of their other services they offer. I looked up XXXX number on their website and contacted them."]))

array([2], dtype=int64)

In [ ]:
from  sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

conf_mat = confusion_matrix(y_test, predictions)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df['product'].values, yticklabels=category_id_df['product'].values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

In [ ]:
import pickle
with open('model.pkl', 'wb') as output_file:
    pickle.dump(model, output_file)

In [ ]:
with open('vectorizer.pkl', 'wb') as output_file:
    pickle.dump(vect, output_file)